<a href="https://colab.research.google.com/github/ftk1000/BERT_demos/blob/master/Copy_of_BertPunc_Inference_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BertPunc_Inference_v2.ipynb**

2020.02.16

##### Edit -> Notebook setting -> hardware accelerator -> GPU,

In [1]:
print('pwd is')
!pwd
print('!ls -ltra shows')
!ls -ltra

pwd is
/content
!ls -ltra shows
total 16
drwxr-xr-x 1 root root 4096 Apr  3 16:24 .
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data
drwxr-xr-x 1 root root 4096 Apr 23 16:11 .config
drwxr-xr-x 1 root root 4096 Apr 28 00:03 ..


#### for the sake of convenience you can create folder in google drive disk,  and put all files from archives to:  C:\Users\USER_NAME\Google Drive\Colab Notebooks\Punctuation_Restoration\For_Farid

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls -l drive/My\ Drive/For_Farid
%cd drive/My\ Drive/For_Farid

total 141332
-rw------- 1 root root     41361 Feb 11 20:48 7k_without_punctuation.txt
-rw------- 1 root root     42578 Feb 12 23:52 7k_with_punctuation.txt
-rw------- 1 root root     27488 Feb 16 22:02 BertPunc.ipynb
-rw------- 1 root root      1034 Feb 11 20:48 convert_to_readable.py
-rw------- 1 root root     10427 Feb 11 20:48 data.py
-rw------- 1 root root      8324 Feb 11 20:48 data.pyc
-rw------- 1 root root 144456098 Feb 11 20:51 Demo-Europarl-EN.pcl
-rw------- 1 root root      6735 Feb 11 20:48 main.py
-rw------- 1 root root       656 Feb 11 20:48 model.py
-rw------- 1 root root     12155 Feb 11 20:48 models.py
-rw------- 1 root root     13876 Feb 11 20:48 models.pyc
-rw------- 1 root root     68553 Feb 11 20:48 Punctuator2.ipynb
-rw------- 1 root root      5944 Feb 11 20:48 punctuator.py
drwx------ 2 root root      4096 Feb 11 20:48 __pycache__
drwx------ 2 root root      4096 Feb 11 20:48 results
-rw------- 1 root root     16827 Feb 11 20:48 timing.docx
/content/drive/My Driv

In [4]:
!ls -ltra

total 141332
drwx------ 2 root root      4096 Feb 11 20:48 results
drwx------ 2 root root      4096 Feb 11 20:48 __pycache__
-rw------- 1 root root      1034 Feb 11 20:48 convert_to_readable.py
-rw------- 1 root root     41361 Feb 11 20:48 7k_without_punctuation.txt
-rw------- 1 root root     16827 Feb 11 20:48 timing.docx
-rw------- 1 root root     13876 Feb 11 20:48 models.pyc
-rw------- 1 root root     12155 Feb 11 20:48 models.py
-rw------- 1 root root       656 Feb 11 20:48 model.py
-rw------- 1 root root     68553 Feb 11 20:48 Punctuator2.ipynb
-rw------- 1 root root      8324 Feb 11 20:48 data.pyc
-rw------- 1 root root      5944 Feb 11 20:48 punctuator.py
-rw------- 1 root root      6735 Feb 11 20:48 main.py
-rw------- 1 root root     10427 Feb 11 20:48 data.py
-rw------- 1 root root 144456098 Feb 11 20:51 Demo-Europarl-EN.pcl
-rw------- 1 root root     42578 Feb 12 23:52 7k_with_punctuation.txt
-rw------- 1 root root     27488 Feb 16 22:02 BertPunc.ipynb


In [5]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from glob import glob
from pytorch_pretrained_bert import BertTokenizer
import torch
from torch import nn
%matplotlib inline
import json
from tqdm import tqdm
from sklearn import metrics
from model import BertPunc
# from data import load_file, preprocess_data, create_data_loader
from torch.utils.data import TensorDataset, DataLoader

In [7]:
print(torch.__version__)

1.4.0


In [8]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
batch_size = 256 # 1024

In [10]:
glob('results/*')

['results/20200206_041031', 'results/20200205_012725']

In [0]:
path = 'results/20200205_012725/'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#with open(path+'hyperparameters.json', 'r') as f:
#    hyperparameters = json.load(f)
hyperparameters = {"segment_size": 32, "dropout": 0.3, "epochs_top": 1, "iterations_top": 2, \
                   "batch_size_top": 1024, "learning_rate_top": 1e-05, "epochs_all": 4, \
                   "iterations_all": 3, "batch_size_all": 256, "learning_rate_all": 1e-05}
punctuation_enc = {
    'O': 0,
    'COMMA': 1,
    'PERIOD': 2,
    'QUESTION': 3
}

segment_size = hyperparameters['segment_size']

100%|██████████| 231508/231508 [00:00<00:00, 313117.09B/s]


In [0]:
def load_file(filename):
    with open(filename, 'r', encoding="utf8", errors="ignore") as f:  # , encoding='utf-8'
        data = f.readlines()
    return data

In [14]:
data_test = load_file('7k_without_punctuation.txt')
print(type(data_test))
print(len(data_test))
print(data_test[:4])

<class 'list'>
1
["To get here we'd spent a day riding motorbikes over a bumpy mountain pass fording knee-high rivers winding our way up switchbacks and even sidestepping a poisonous snake Now we were close to our destination a black cardamom forest on a nearby peak but couldn't find the approach trail among the shrubs and wildflowers Lang's husband Duong had just wandered off to look for it As it turned out Lang and the hunter were from the same village not far from Hoang Lien National Park He had been farming cardamom in the park for years he said and knew exactly where her family was camped We had entered the park a collection of rugged mountains and valleys near Vietnam's border with China to see cardamom being harvested in the wild Giang Thi Lang and Nguyen Danh Duong are trekking guides in the nearby town of Sa Pa I had befriended them years earlier while living in Hanoi the Vietnamese capital Lang's family has cultivated cardamom in the Hoang Lien Mountains since the 1990s and n

In [15]:
X = []
for line in data_test:
    # word = line.split()
    # print(word)
    tokens = tokenizer.tokenize(line)
    print(len(tokens))
    print(line[:10])
    x = tokenizer.convert_tokens_to_ids(tokens)
    if len(x) > 0:
        X += x
print(X[:10])
print(len(X))

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (8112 > 512). Running this sequence through BERT will result in indexing errors


8112
To get her
[2000, 2131, 2182, 2057, 1005, 1040, 2985, 1037, 2154, 5559]
8112


In [16]:
print(' Original: ', data_test)
print('Tokenized: ', tokens)
print('Token IDs: ', X)

 Original:  ["To get here we'd spent a day riding motorbikes over a bumpy mountain pass fording knee-high rivers winding our way up switchbacks and even sidestepping a poisonous snake Now we were close to our destination a black cardamom forest on a nearby peak but couldn't find the approach trail among the shrubs and wildflowers Lang's husband Duong had just wandered off to look for it As it turned out Lang and the hunter were from the same village not far from Hoang Lien National Park He had been farming cardamom in the park for years he said and knew exactly where her family was camped We had entered the park a collection of rugged mountains and valleys near Vietnam's border with China to see cardamom being harvested in the wild Giang Thi Lang and Nguyen Danh Duong are trekking guides in the nearby town of Sa Pa I had befriended them years earlier while living in Hanoi the Vietnamese capital Lang's family has cultivated cardamom in the Hoang Lien Mountains since the 1990s and now he

In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-((segment_size-1)//2-1):]+x+x[:segment_size//2]

    for i in range(len(x_pad)-segment_size+2):
        segment = x_pad[i:i+segment_size-1]
        segment.insert((segment_size-1)//2, 0)
        X.append(segment)

    return np.array(X)

In [0]:
X_zero_halfway = insert_target(x, segment_size)

In [19]:
print(type(X_zero_halfway))
X_zero_halfway.shape

<class 'numpy.ndarray'>


(8112, 32)

In [0]:
# convert to torch's internal format
data_set = TensorDataset(torch.from_numpy(X_zero_halfway).long())

In [21]:
print(type(data_set))

<class 'torch.utils.data.dataset.TensorDataset'>


what is a batch? we think it is a matrix of size 1024x32, each entry is a code-word

In [22]:
print(batch_size)

256


In [0]:
data_loader = DataLoader(data_set, batch_size, shuffle=False)

In [24]:
type(data_loader)

torch.utils.data.dataloader.DataLoader

In [25]:
print(punctuation_enc)

{'O': 0, 'COMMA': 1, 'PERIOD': 2, 'QUESTION': 3}


In [0]:
output_size = len(punctuation_enc)

In [0]:
dropout = hyperparameters['dropout']

In [28]:
# from torch import nn
# from model import BertPunc
bert_punc = nn.DataParallel(BertPunc(segment_size, output_size, dropout).cuda())

100%|██████████| 407873900/407873900 [00:36<00:00, 11263975.87B/s]


In [29]:
BertPunc

model.BertPunc

In [31]:
type(BertPunc)

type

In [32]:
# what is this doing?????

bert_punc.load_state_dict(torch.load(path+'model'))

<All keys matched successfully>

In [0]:
# evaluate parallelized Bert_Punc
bert_punc.eval();

In [34]:
for batch in data_loader:
    # print(batch)
    print(type(batch))
    # print(batch[0])
    print(type(batch[0]))
    break

<class 'list'>
<class 'torch.Tensor'>


In [35]:
import time
start_time = time.time()

pred = []
for inputs in tqdm(data_loader):
    with torch.no_grad():
        inputs = inputs[0].cuda()
        output = bert_punc(inputs)
        pred += list(output.argmax(dim=1).cpu().data.numpy().flatten())

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

100%|██████████| 32/32 [00:10<00:00,  3.03it/s]

00:00:10


In [36]:
print(pred[:20])

[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [37]:
pred.count(0)

7139

In [38]:
pred.count(1)

516

In [39]:
pred.count(2)

456

In [0]:
# [to be continued]